# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA
* Model: BERT ("distilbert-base-uncased")
* Evaluation approach: Hugging Face Trainer's Evaluate Method
* Fine-tuning dataset: "dair-ai/emotion"

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from datasets import load_dataset


splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}

print(ds['train'])

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched = True
    )


In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels = 6
)

for param in model.base_model.parameters():
    param.requires_grad = True

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [6]:
trainer.evaluate()

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 1.8424593210220337,
 'eval_model_preparation_time': 0.001,
 'eval_accuracy': 0.0665,
 'eval_runtime': 3.1928,
 'eval_samples_per_second': 626.402,
 'eval_steps_per_second': 156.601}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels = 6)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [8]:
from peft import LoraConfig
config = LoraConfig()

In [9]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [10]:
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,781,766 || trainable%: 0.2686


In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="steps_per_second"
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [21]:
lora_trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

{'loss': 1.5794, 'grad_norm': 0.15350928902626038, 'learning_rate': 0.0018000000000000002, 'epoch': 0.5}
{'loss': 1.585, 'grad_norm': 0.10244404524564743, 'learning_rate': 0.0016, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_runtime': 1.4878, 'eval_samples_per_second': 1344.236, 'eval_steps_per_second': 84.015, 'epoch': 1.0}
{'loss': 1.5769, 'grad_norm': 0.13951750099658966, 'learning_rate': 0.0014, 'epoch': 1.5}
{'loss': 1.5864, 'grad_norm': 0.12012731283903122, 'learning_rate': 0.0012, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_runtime': 1.4783, 'eval_samples_per_second': 1352.934, 'eval_steps_per_second': 84.558, 'epoch': 4.0}
{'loss': 1.5779, 'grad_norm': 0.08462361246347427, 'learning_rate': 0.0002, 'epoch': 4.5}
{'loss': 1.577, 'grad_norm': 0.09688691049814224, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_runtime': 1.4677, 'eval_samples_per_second': 1362.655, 'eval_steps_per_second': 85.166, 'epoch': 5.0}
{'train_runtime': 156.8568, 'train_samples_per_second': 510.019, 'train_steps_per_second': 31.876, 'train_loss': 1.5801047119140625, 'epoch': 5.0}


TrainOutput(global_step=5000, training_loss=1.5801047119140625, metrics={'train_runtime': 156.8568, 'train_samples_per_second': 510.019, 'train_steps_per_second': 31.876, 'total_flos': 1938771154706112.0, 'train_loss': 1.5801047119140625, 'epoch': 5.0})

In [23]:
lora_model.save_pretrained("lora-bert")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [25]:
from peft import AutoPeftModelForCausalLM
lora_model = AutoPeftModelForCausalLM.from_pretrained("lora-bert", num_labels=6)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [26]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [27]:
trainer.evaluate()

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 1.7520630359649658,
 'eval_model_preparation_time': 0.003,
 'eval_accuracy': 0.2905,
 'eval_runtime': 5.4918,
 'eval_samples_per_second': 364.177,
 'eval_steps_per_second': 91.044}

From this we see the before evaluation accuracy of 0.0665

{'eval_loss': 1.8424593210220337,
 'eval_model_preparation_time': 0.001,
 'eval_accuracy': 0.0665,
 'eval_runtime': 3.1928,
 'eval_samples_per_second': 626.402,
 'eval_steps_per_second': 156.601}

---

and after training through using LORA PEFT, we get an accuracy of: 0.2905

{'eval_loss': 1.7520630359649658,
 'eval_model_preparation_time': 0.003,
 'eval_accuracy': 0.2905,
 'eval_runtime': 5.4918,
 'eval_samples_per_second': 364.177,
 'eval_steps_per_second': 91.044}